<a href="https://colab.research.google.com/github/azernik/semeval_2025_task1/blob/main/sentence_type_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install libaries & Load dataset

In [ ]:
!pip install -q datasets  # error is ok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Load the MAGPIE dataset
dataset = load_dataset("gsarti/magpie", trust_remote_code=True)
df_magpie = pd.DataFrame(dataset["train"])

len(df_magpie)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.97k [00:00<?, ?B/s]

magpie.py:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

magpie.tsv:   0%|          | 0.00/15.7M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

44451

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'annotation', 'idiom', 'usage', 'variant', 'pos_tags'],
        num_rows: 44451
    })
})

### Load ADMIRE train dataset

In [ ]:
# for downloading the train zip from Drive
!pip install -q gdown

# install clip
!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import gdown

# download taskA file from Adam's Drive (public) and unzip
file_id = "105JdQU_u98w_xSYaNNSj-r4RsyTPXZEF"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "taskA.zip", quiet=True)
! unzip -q - taskA.zip

In [ ]:
import os
import pandas as pd

# define locations
taska_folder = "train"
taska_tsv_filename = "subtask_a_train.tsv"

# load data
df = pd.read_csv(f"{taska_folder}/{taska_tsv_filename}", delimiter="\t")

In [ ]:
from ast import literal_eval

# set image paths in df
image_name_cols = ['image1_name', 'image2_name', 'image3_name', 'image4_name', 'image5_name']
df['compound_dir'] = df["compound"].apply(lambda nc: nc.replace("'", "_"))
df['image_paths'] = df.apply(lambda row: [os.path.join(taska_folder, row['compound_dir'], row[image_name]) for image_name in image_name_cols], axis=1)
df['image_label_map'] = df.apply(lambda row: {row[name]: i for i, name in enumerate(image_name_cols)}, axis=1)

# create column with readable expected image order, e.g. [3, 1, 5, 2, 4]
df['expected_order_readable'] = df.apply(lambda row: [row['image_label_map'][name] for name in literal_eval(row['expected_order'])], axis=1)

### Preprocessing

In [ ]:
# map sentence type to binary labels (0 = literal, 1 = figurative)
df_magpie["label"] = df_magpie["usage"].map({"literal": 0, "figurative": 1})

# split into train, validation, and test sets
train_df, temp_df = train_test_split(df_magpie, test_size=0.2, random_state=42, stratify=df_magpie["label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

# check class distribution
print("Train label distribution:", train_df["label"].value_counts())
print("Validation label distribution:", val_df["label"].value_counts())
print("Test label distribution:", test_df["label"].value_counts())


Train label distribution: label
1    27310
0     8250
Name: count, dtype: int64
Validation label distribution: label
1    3414
0    1031
Name: count, dtype: int64
Test label distribution: label
1    3414
0    1032
Name: count, dtype: int64


In [ ]:
import torch
from transformers import AutoTokenizer

# load bert's tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# tokenize the sentences
def tokenize_sentences(df, tokenizer, max_length=128):
    return tokenizer(
        list(df["sentence"]),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    )

train_encodings = tokenize_sentences(train_df, tokenizer)
val_encodings = tokenize_sentences(val_df, tokenizer)
test_encodings = tokenize_sentences(test_df, tokenizer)

train_labels = torch.tensor(train_df["label"].values)
val_labels = torch.tensor(val_df["label"].values)
test_labels = torch.tensor(test_df["label"].values)


In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load pre-trained model
config = AutoConfig.from_pretrained("bert-base-uncased", hidden_dropout_prob=0.3, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", config=config)
model = model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 16

# create Datasets and DataLoaders
train_dataset = TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels)
val_dataset = TensorDataset(val_encodings["input_ids"], val_encodings["attention_mask"], val_labels)
test_dataset = TensorDataset(test_encodings["input_ids"], test_encodings["attention_mask"], test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.nn as nn

# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)

# Convert to tensor for PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

### Training loop

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm


# hyperparams
num_epochs = 4

# define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(train_loader)
num_warmup_steps = int(0.1 * num_training_steps)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# Training loop
for epoch in range(num_epochs):

    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        inputs, attention_masks, labels = [x.to(device) for x in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=inputs, attention_mask=attention_masks, labels=labels)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # save the model at the end of each epoch
    model_save_path = f"model_epoch_{epoch+1}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

    # validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(val_loader):
            inputs, attention_masks, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=inputs, attention_mask=attention_masks)
            logits = outputs.logits
            loss = loss_fn(logits, labels)
            val_loss += loss.item()
            preds = logits.argmax(dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader)
    accuracy = correct / total
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")


100%|██████████| 2223/2223 [06:10<00:00,  6.01it/s]


Model saved to model_epoch_1.pth


100%|██████████| 278/278 [00:14<00:00, 19.63it/s]


Epoch 1: Train Loss = 0.0858, Val Loss = 0.2051


100%|██████████| 2223/2223 [06:10<00:00,  6.00it/s]


...

Epoch 3: Train Loss = 0.0403, Val Loss = 0.2454


100%|██████████| 2223/2223 [06:10<00:00,  6.00it/s]


Model saved to model_epoch_4.pth


100%|██████████| 278/278 [00:14<00:00, 19.61it/s]

Epoch 4: Train Loss = 0.0226, Val Loss = 0.2206


In [ ]:
from google.colab import files
files.download('model_epoch_2.pth')
files.download('model_epoch_3.pth')
files.download('model_epoch_4.pth')

### Evaluation on test split

In [ ]:
# load 'model_epoch_2.pth'
model.load_state_dict(torch.load('model_epoch_4.pth'))
model.save_pretrained("sentence_type_classifier")

<ipython-input-14-af3cb090dbf3>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_epoch_4.pth'))


In [ ]:
from sklearn.metrics import classification_report

# Put the model in evaluation mode
model.eval()

test_loss = 0
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs, attention_masks, labels = [x.to(device) for x in batch]
        outputs = model(input_ids=inputs, attention_mask=attention_masks)
        logits = outputs.logits
        loss = loss_fn(logits, labels)  # Use the same loss function as training
        test_loss += loss.item()

        preds = logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        # Save predictions and true labels for metrics
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_loss /= len(test_loader)  # Average test loss
accuracy = correct / total  # Test accuracy

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Calculate precision, recall, F1 score, and support
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["literal", "figurative"]))


100%|██████████| 278/278 [00:14<00:00, 19.84it/s]

Test Loss: 0.1990
Test Accuracy: 0.9417

Classification Report:
              precision    recall  f1-score   support

     literal       0.83      0.94      0.88      1032
  figurative       0.98      0.94      0.96      3414

    accuracy                           0.94      4446
   macro avg       0.91      0.94      0.92      4446
weighted avg       0.95      0.94      0.94      4446



### Evaluation on ADMIRE dataset

In [ ]:
# Save the model
# model.save_pretrained("sentence_type_classifier")

# Load during inference
from transformers import AutoModelForSequenceClassification

classifier = AutoModelForSequenceClassification.from_pretrained("sentence_type_classifier").to(device)

def predict_sentence_type(sentence, tokenizer, classifier):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = classifier(**inputs).logits
    return "figurative" if logits.argmax(dim=-1).item() == 1 else "literal"

In [ ]:
sentence_type_df = df[['sentence', 'compound', 'sentence_type']].copy()

In [ ]:
sentence_type_df

,sentence,compound,sentence_type
0,It took a lot of elbow grease to get the old e...,elbow grease,idiomatic
1,"It's a constant battle for us, as he is a morn...",night owl,idiomatic
2,Even the somewhat seedy failed private eye has...,heart of gold,idiomatic
3,ESA's Space Weather Office is like Europe's st...,agony aunt,idiomatic
4,"This aged, rich wine is no shrinking violet wi...",shrinking violet,idiomatic
...,...,...,...
65,"When you're serving cold turkey, take out only...",cold turkey,literal
66,Howard said there was a lot of low-hanging fru...,low-hanging fruit,idiomatic
67,There's no need to worry if you're a busy bee ...,busy bee,idiomatic
68,Wet blankets draped over neighboring cabins pr...,wet blanket,literal


In [ ]:
sentence_type_df['predicted_sentence_type'] = sentence_type_df['sentence'].apply(lambda x: predict_sentence_type(x, tokenizer, classifier))

,compound,subset,sentence_type,sentence,expected_order,image1_name,image1_caption,image2_name,image2_caption,image3_name,image3_caption,image4_name,image4_caption,image5_name,image5_caption,compound_dir,image_paths,image_label_map,expected_order_readable,paraphrased_nc
0,elbow grease,Train,idiomatic,It took a lot of elbow grease to get the old e...,"['35234427395.png', '53378381715.png', '399382...",35234427395.png,The image depicts a hand holding a sponge and ...,39938261459.png,The image depicts a hand wearing a yellow work...,53378381715.png,The image depicts a hand holding a duster with...,54879908369.png,The image depicts a person wearing knee pads a...,74852536462.png,The image depicts a person wearing a black out...,elbow grease,"[train/elbow grease/35234427395.png, train/elb...","{'35234427395.png': 0, '39938261459.png': 1, '...","[0, 2, 1, 4, 3]",Manual labor or physical effort put into a tas...
1,night owl,Train,idiomatic,"It's a constant battle for us, as he is a morn...","['61697797701.png', '93189810779.png', '893752...",00982495584.png,The image depicts a nighttime scene with a lar...,61697797701.png,The image depicts a cartoon-style illustration...,89375227504.png,The image depicts a cartoon-style owl perched ...,93189810779.png,The image depicts a cartoon-style illustration...,93541983868.png,"The image depicts a dumbbell, which is a commo...",night owl,"[train/night owl/00982495584.png, train/night ...","{'00982495584.png': 0, '61697797701.png': 1, '...","[1, 3, 2, 0, 4]",A person prone to staying awake and active at ...
2,heart of gold,Train,idiomatic,Even the somewhat seedy failed private eye has...,"['86137977215.png', '78062290185.png', '542405...",54240592941.png,"The image depicts a large, metallic safe with ...",78062290185.png,The image depicts a joyful scene featuring a y...,86137977215.png,The image depicts a cartoon scene where a youn...,90660547751.png,"The image depicts a futuristic, stylized space...",92088849364.png,"The image depicts a stylized, artistic represe...",heart of gold,"[train/heart of gold/54240592941.png, train/he...","{'54240592941.png': 0, '78062290185.png': 1, '...","[2, 1, 0, 4, 3]","Refers to someone who is naturally kind, compa..."
3,agony aunt,Sample,idiomatic,ESA's Space Weather Office is like Europe's st...,"['83600499282.png', '57658144685.png', '025128...",02512838127.png,The image depicts a serene outdoor scene featu...,32964421720.png,The image depicts a cartoon-style illustration...,57658144685.png,The image depicts a scene of a person sitting ...,83600499282.png,"The image depicts a person sitting at a desk, ...",92533456778.png,The image depicts a cartoon character of a wom...,agony aunt,"[train/agony aunt/02512838127.png, train/agony...","{'02512838127.png': 0, '32964421720.png': 1, '...","[3, 2, 0, 1, 4]",A columnist or figure offering advice to other...
4,shrinking violet,Train,idiomatic,"This aged, rich wine is no shrinking violet wi...","['77861539717.png', '68016869942.png', '118443...",11844321898.png,The image depicts a bouquet of purple tulips a...,45394842176.png,"The image depicts a magnifying glass, commonly...",68016869942.png,"The image depicts a small, cute puppy standing...",77861539717.png,The image depicts an animated character with c...,97482048489.png,The image is a colorful and vibrant illustrati...,shrinking violet,"[train/shrinking violet/11844321898.png, train...","{'11844321898.png': 0, '45394842176.png': 1, '...","[3, 2, 0, 1, 4]","A shy, often overshadowed individual, preferri..."


In [ ]:
import torch
from sklearn.metrics import classification_report
from transformers import AutoTokenizer

# Load the SemEval data
sentence_type_df["label"] = sentence_type_df["sentence_type"].map({"literal": 0, "idiomatic": 1})

# Tokenize the SemEval sentences
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_sentences(df, tokenizer, max_length=128):
    return tokenizer(
        list(df["sentence"]),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Tokenize the SemEval data
semeval_encodings = tokenize_sentences(sentence_type_df, tokenizer)

# Create PyTorch tensors for the SemEval data
semeval_inputs = semeval_encodings["input_ids"].to(device)
semeval_attention_masks = semeval_encodings["attention_mask"].to(device)
semeval_labels = torch.tensor(sentence_type_df["label"].values).to(device)

# Put the model in evaluation mode
model.eval()

# Perform inference
with torch.no_grad():
    outputs = model(input_ids=semeval_inputs, attention_mask=semeval_attention_masks)
    predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

sentence_type_df['predicted_label'] = predictions

# Calculate metrics
true_labels = semeval_labels.cpu().numpy()
print("Classification Report:")
print(classification_report(true_labels, predictions, target_names=["Literal", "Figurative"]))


Classification Report:
              precision    recall  f1-score   support

     Literal       0.64      0.97      0.77        31
  Figurative       0.96      0.56      0.71        39

    accuracy                           0.74        70
   macro avg       0.80      0.77      0.74        70
weighted avg       0.82      0.74      0.74        70



In [ ]:
sentence_type_df['is_correct'] = sentence_type_df['label'] == sentence_type_df['predicted_label']

In [ ]:
sentence_type_df.to_csv('sentence_type_predictions.csv', index=False)